In [1]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeCusco
from qiskit import transpile

import numpy as np
from datetime import datetime

In [2]:
# Load saved credentials
service = QiskitRuntimeService()

## real ibm device

In [ ]:
def get_count_measx_prepy(prep:int, shots:int, repetition:int, qc, backend) -> int:
    error_count = 0
    
    for rep in range(repetition):
        if prep:
            qc.x(0)
        qc.measure_all()
            
        isa_circuit = pass_manager.run(qc)
            
        sampler = Sampler(real_backend)
        job = sampler.run([isa_circuit], shots = shots)
        result = job.result()[0]
        counts = result['__value__']['data'].meas.get_counts()
        if '0' not in counts.keys():
            counts['0'] = 0
        if '1' not in counts.keys():
            counts['1'] = 0
        
        error_count = error_count + counts

    # if statistics such as std, min, or max are requested??: return type should be modified
    
    return error_count

def get_readout_assignment_error(shots:list[int], repetition:list[int], qubit_index, backend) -> float:
    readout_assignment_error = 0

    qr = QuantumRegister(1)
    qc = QuantumCircuit(qr)

    error_count = [0,0] # [meas1prep0, meas0prep1]
    for bit in range(2):
        error_count[bit] = get_count_measx_prepy(prep? = bit, shots[bit], repetition[bit], backend_name, qubit_index)
    
#    if numbers of preparation differ?: #do not consider now...
#        readout_assignment_error = sum(error_count) / sum(shots)

    return readout_assignment_error

In [ ]:
real_backend = service.least_busy()
#pass_manager = generate_preset_pass_manager(optimization_level=0, backend=backend, initial_layout = [43,42,34,44,41,24,45]) # initial_layout=[], layout_method='trivial'

backend_prop = real_backend.properties().to_dict()
initial_layout = [62,61,72,63,60,81,64]

shots = 5000

RR = 5

num_target = 0

print("chosen device:",backend_prop['backend_name'])
print("chosen qubits:", initial_layout[0:num_target+1])
print("#shots:",shots,"#repetition:",RR)

#distilled_results = np.zeros((RR,M*N,3))
real_all_counts = [{'pad':0}]

pass_manager = generate_preset_pass_manager(optimization_level=0, backend=real_backend, initial_layout = [initial_layout[0]])
for rr in range(RR):

    qr = QuantumRegister(1)
    
    qc = QuantumCircuit(qr)

    qc.measure_all()
    
    isa_circuit = pass_manager.run(qc)
    
    sampler = Sampler(real_backend)
    job = sampler.run([isa_circuit], shots = shots)
    result = job.result()[0]
    ##print(result['__value__']['data'].meas.get_counts())
    counts = result['__value__']['data'].meas.get_counts()
    if '0' not in counts.keys():
        counts['0'] = 0
    if '1' not in counts.keys():
        counts['1'] = 0
    
    real_all_counts.append(counts)
    
    #print(rr)#, 'counts: ', counts)
        
real_all_counts = real_all_counts[1:]
print('prep0:')
for cc in real_all_counts:
    print(cc)


real_all_counts_x = [{'pad':0}]

pass_manager = generate_preset_pass_manager(optimization_level=0, backend=real_backend, initial_layout = [initial_layout[0]])
for rr in range(RR):

    qr = QuantumRegister(1)
    
    qc = QuantumCircuit(qr)

    qc.x(0)
    
    qc.measure_all()
    
    isa_circuit = pass_manager.run(qc)
    
    sampler = Sampler(real_backend)
    job = sampler.run([isa_circuit], shots = shots)
    result = job.result()[0]
    ##print(result['__value__']['data'].meas.get_counts())
    counts = result['__value__']['data'].meas.get_counts()
    if '0' not in counts.keys():
        counts['0'] = 0
    if '1' not in counts.keys():
        counts['1'] = 0
    
    real_all_counts_x.append(counts)
    
    #print(rr)#, 'counts: ', counts)
        
real_all_counts_x = real_all_counts_x[1:]
print('prep1:')
for cc in real_all_counts_x:
    print(cc)

chosen device: ibm_strasbourg
chosen qubits: [62]
#shots: 5000 #repetition: 5
prep0:
{'0': 4860, '1': 140}
{'0': 4882, '1': 118}
{'0': 4877, '1': 123}
{'0': 4867, '1': 133}
{'0': 4893, '1': 107}
prep1:
{'1': 4726, '0': 274}
{'1': 4748, '0': 252}
{'1': 4723, '0': 277}
{'1': 4750, '0': 250}
{'1': 4740, '0': 260}


### statistical analysis

In [26]:
#for cc in sim_all_counts:
#    print(cc['00']/shots,cc['10']/shots,cc['01']/shots,cc['11']/shots)

real_prob_data = {}
real_prob_data_x = {}

probs = {'0':0,'1':0}
probs_x = {'0':0,'1':0}
for rr in range(RR):
    for outcome in probs.keys():
        probs[outcome] += real_all_counts[rr][outcome[-1::-1]] / RR / shots
        probs_x[outcome] += real_all_counts_x[rr][outcome[-1::-1]] / RR / shots
#real_prob_data.append(probs)
real_prob_data[initial_layout[0]] = probs
real_prob_data_x[initial_layout[0]] = probs_x
        
    
print("big-endian encoding") # IBM qn ... q1 q0 => q0 q1 ... qn
for qubit, freq in real_prob_data.items():
    print(qubit, freq)
for qubit, freq in real_prob_data_x.items():
    print(qubit, freq)

big-endian encoding
62 {'0': 0.97516, '1': 0.02484}
62 {'0': 0.05252, '1': 0.9474800000000001}


### Backend Properties

In [29]:
print("Backend name:", backend_prop['backend_name'])
print("Last calibrated date:", backend_prop['last_update_date'])
for qq in range(num_target+1):
    print("========================================================================================")
    print("qubit",initial_layout[qq],"| readout_error (%):", backend_prop['qubits'][initial_layout[qq]][4]['value'])
    print("qubit",initial_layout[qq],"| prep0_meas1 (%):", backend_prop['qubits'][initial_layout[qq]][6]['value'])
    print("qubit",initial_layout[qq],"| prep1_meas0 (%):", backend_prop['qubits'][initial_layout[qq]][5]['value'])
    print("qubit",backend_prop['gates'][initial_layout[qq]]['qubits'][0],"| id_error (%):", backend_prop['gates'][initial_layout[qq]]['parameters'][0]['value'])
    print("qubit",backend_prop['gates'][initial_layout[qq]+127]['qubits'][0],"| rz_error (%):", backend_prop['gates'][initial_layout[qq]+127]['parameters'][0]['value'])
    print("qubit",backend_prop['gates'][initial_layout[qq]+127*2]['qubits'][0],"| sx_error (%):", backend_prop['gates'][initial_layout[qq]+127*2]['parameters'][0]['value'])
    print("qubit",backend_prop['gates'][initial_layout[qq]+127*2+127]['qubits'][0],"| x_error (%):", backend_prop['gates'][initial_layout[qq]+127*2+127]['parameters'][0]['value'])

    if qq > 0:
        for ecr_spec in backend_prop['gates'][127*4:]:
            if (initial_layout[0] in ecr_spec['qubits']) and (initial_layout[qq] in ecr_spec['qubits']):
                print("qubits", ecr_spec['qubits'],"| ecr_error (%):",ecr_spec['parameters'][0]['value'])
                
# 'qubits' : [qubit_idx][T1(us),T2(us),frequency(GHz),anharmonicity(GHz),readout_error,prob_meas0_prep1,prob_meas1_prep0,readout_length(ns)]{'date','name','unit','value'}
# 'gates' : [id,rz,sx,x,ecr]{'qubits','gate','parameters'[0]{'name','value'}}

Backend name: ibm_strasbourg
Last calibrated date: 2024-11-04 10:58:07+09:00
qubit 62 | readout_error (%): 0.050899999999999945
qubit 62 | prep0_meas1 (%): 0.047
qubit 62 | prep1_meas0 (%): 0.05479999999999996
qubit 62 | id_error (%): 0.0003977075415263361
qubit 62 | rz_error (%): 0
qubit 62 | sx_error (%): 0.0003977075415263361
qubit 62 | x_error (%): 0.0003977075415263361


## Solve equations to get error parameters

The probabilities from measurement experiments can be seen as functions of three error parameters $f,q$ and $\epsilon$. So, we can solve the series of equations for known probabilities $p(ij)$.

$$p(00) = (1-\epsilon)\left[f^2 (1-q)^2 + (1-f)^2 q^2 + f(1-f)(1-q)q  + (1-f)^2 q(1-q) + (1-f)fq^2\right] + \epsilon/4 $$
$$p(01) = (1-\epsilon)\left[f^2 (1-q)q + (1-f)^2 q^2 + f(1-f)(1-q)^2  + (1-f)^2 q^2 + (1-f)fq(1-q)\right] + \epsilon/4 $$
$$p(10) = (1-\epsilon)\left[f^2 q(1-q) + (1-f)^2 (1-q)q + f(1-f)q^2  + (1-f)^2 (1-q)^2 + (1-f)f(1-q)q\right] + \epsilon/4 $$
$$p(11) = (1-\epsilon)\left[f^2 q^2 + (1-f)^2 (1-q)^2 + f(1-f) q(1-q)  + (1-f)^2 (1-q)q + (1-f)f(1-q)^2\right] + \epsilon/4 $$

In [63]:
from scipy.optimize import root

def equations(X,P):
    return [(1-X[2])*((X[0]**2)*(X[1]**2)+((1-X[0])**2)*((1-X[1])**2)+X[0]*(1-X[0])*X[1]*(1-X[1])+((1-X[0])**2)*(1-X[1])*X[1]+(1-X[0])*X[0]*((1-X[1])**2))+X[2]/4-P['11'],
        (1-X[2])*((X[0]**2)*(1-X[1])*X[1]+((1-X[0])**2)*(1-X[1])*X[1]+X[0]*(1-X[0])*(X[1]**2)+((1-X[0])**2)*((1-X[1])**2)+(1-X[0])*X[0]*(1-X[1])*X[1])+X[2]/4-P['10'],
        (1-X[2])*((X[0]**2)*(1-X[1])*X[1]+((1-X[0])**2)*(X[1]**2)+X[0]*(1-X[0])*((1-X[1])**2)+((1-X[0])**2)*(X[1]**2)+(1-X[0])*X[0]*(1-X[1])*X[1])+X[2]/4-P['01']]#,
        #(1-X[2])*((X[0]**2)*((1-X[1])**2)+((1-X[0])**2)*(X[1]**2)+X[0]*(1-X[0])*(1-X[1])*X[1]+((1-X[0])**2)*X[1]*(1-X[1])+(1-X[0])*X[0]*(X[1]**2))+X[2]/4-P['00']]

for qubit, freq in real_prob_data.items():
    #probs = [freq['00'],freq['01'],freq['10'],freq['11']]
    print(qubit, freq)
    solution = root(equations, [0,0,0], args=freq)
    print("(1-f,q,e) =", solution.x)


61 {'00': 0.9835333333333335, '01': 0.005913333333333332, '10': 0.006973333333333332, '11': 0.003579999999999999}
(1-f,q,e) = [ 0.00796449 -0.99449949  1.14254831]
72 {'00': 0.9176600000000001, '01': 0.07110666666666667, '10': 0.0077800000000000005, '11': 0.003453333333333333}
(1-f,q,e) = [ 0.01031037 -0.83577618  1.15734448]
63 {'00': 0.9565999999999999, '01': 0.03078, '10': 0.007346666666666665, '11': 0.005273333333333332}
(1-f,q,e) = [0.11398513 0.49085597 2.1982331 ]


### Results from MATLAB.fsolve

System qubit = 62,

target qubit = 61: $(1-f,q,\epsilon) = (-0.0011, 0.0024, 0.0185)$

target qubit = 72: $(1-f,q,\epsilon) = (0.0648, 0.0692, -0.3035)$

target qubit = 63: $(1-f,q,\epsilon) = (0.0240, 0.0262, -0.0805)$